In [ ]:
import geopandas as gpd
import pandas as pd
from plotly.colors import sequential
from src.db_helpers import get_cases_by_date
from src.plotting_helpers import get_color_for_val
import urllib.request, json 

pd.options.plotting.backend = 'plotly'

cases = get_cases_by_date('2020-06-30')

def get_color(val):
    if val in cases:
        return get_color_for_val(cases[val], 0, cases.max(), sequential.YlOrRd), 0.8
    else:
        return 'white', 0.0

In [ ]:
with urllib.request.urlopen('https://opendata.arcgis.com/datasets/41c3a7bd375547069a78fce90153cbc0_5.geojson') as url:
    zips = json.loads(url)

In [ ]:
from ipywidgets import Output

out = Output()

@out.capture()
def style_zip(feature):
    color, opacity = get_color(int(feature['properties']['ZIP']))
    return {
        'weight': 0.8,
        'color': 'white',
        'fillOpacity': opacity,
        'fillColor': color,
    }
    
out

In [ ]:
from ipyleaflet import Map, GeoJSON, basemaps, WidgetControl
from ipywidgets import Text, HTML

m = Map(basemap=basemaps.OpenStreetMap.BlackAndWhite, center=(33.0,-116),zoom=9)

zips_layer = GeoJSON(data=zips,
                     hover_style={'color': 'black', 'opacity': 0.9},
                     style_callback=style_zip)
m.add_layer(zips_layer)

html = HTML('''Move the cursor over an area''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

def update_html(feature,  **kwargs):
    zip_code = feature['properties']['ZIP']
    cases_in_zip = 0
    if zip_code in cases:
        cases_in_zip = cases[zip_code]
    html.value = '''
        <strong>{}</strong><br>{}<br>{:.0f}
    '''.format(zip_code,
               feature['properties']['COMMUNITY'],
               cases_in_zip)

zips_layer.on_hover(update_html)

m